# Создание целевой переменной

#### Важно: целевая переменная создается без привязки по времени. Для экспериментов с временными рядами смотреть ноутбук time_series_analysis.ipynb

In [1]:
import pandas as pd
from olist_churn_prediction.paths import SRC_DIR, PROCESSED_DIR, INTERIM_NOTEBOOK_DIR

Возьмем за классы метки '1' - ушел и '0' - не ушел. За условие ухода возьмем факт отсутствия покупок за следующие 120 дней с последней покупки. Так как данные за 2016-2018 годы, то за reference-дату возьмем последнее доступное `order_purchase_timestamp`: это даст нам настоящую метку ухода (если взять 2025 год за последнюю дату, то все клиенты будут считаться ушедшими).

In [2]:
df = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "merged_df.parquet")

In [3]:
reference_date = df['order_purchase_timestamp'].max()
last_orders = df.groupby('customer_id')['order_purchase_timestamp'].max().reset_index()
last_orders['days_since_last_order'] = (reference_date - last_orders['order_purchase_timestamp']).dt.days
last_orders['churned'] = (last_orders['days_since_last_order'] > 120).astype(int)
df = df.merge(last_orders[['customer_id', 'churned']], on='customer_id', how='left')

In [4]:
print(reference_date)

2018-09-28 10:54:31


In [5]:
last_orders.head()

,customer_id,order_purchase_timestamp,days_since_last_order,churned
0,0000ca4ff2795842fd6b122c6d974468,2018-01-27 10:04:53,244,1
1,00010f206878ba01f199aec4237de72e,2017-09-10 14:35:08,382,1
2,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,317,1
3,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,439,1
4,00017f1d6cce0d56046219a2cfabcbbb,2017-08-19 16:22:14,404,1


In [6]:
last_orders['days_since_last_order'].describe()

count    93882.000000
mean       308.885708
std        157.080852
min          0.000000
25%        190.000000
50%        307.000000
75%        430.000000
max        753.000000
Name: days_since_last_order, dtype: float64

In [10]:
last_orders['churned'].head()

0    1
1    1
2    1
3    1
4    1
Name: churned, dtype: int64

In [11]:
last_orders['churned'].value_counts()

churned
1    81024
0    12858
Name: count, dtype: int64

Важное замечание: классы несбалансированы. Лочически соотношение выглядит приемлемым, оставим условие в 120 дней.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93882 entries, 0 to 93881
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       93882 non-null  string        
 1   order_products_value           93882 non-null  float32       
 2   order_freight_value            93882 non-null  float32       
 3   order_items_qty                93882 non-null  Int64         
 4   product_photos_qty             93882 non-null  Int64         
 5   product_name_lenght            93882 non-null  Float64       
 6   product_description_lenght     93882 non-null  Float64       
 7   review_creation_date           93882 non-null  datetime64[ns]
 8   review_answer_timestamp        93882 non-null  datetime64[ns]
 9   order_purchase_timestamp       93882 non-null  datetime64[ns]
 10  order_aproved_at               93882 non-null  datetime64[ns]
 11  order_estimated

In [ ]:
df.to_parquet(INTERIM_NOTEBOOK_DIR / "merged_churn_interim.parquet")